# Lab 2

In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [2]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!ls /users/trush/CSC496/PageRank/Lab2

auth_log  Lab2.ipynb


In [17]:
auth_log = sc.textFile("/users/trush/CSC496/PageRank/Lab2/auth_log/auth.log")
auth_log.take(50)

['Dec 13 00:06:16 submitty sshd[19509]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.184  user=root',
 'Dec 13 00:06:17 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:06:21 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:07:01 submitty CRON[19523]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Dec 13 00:07:02 submitty CRON[19523]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Dec 13 00:08:01 submitty CRON[19536]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Dec 13 00:08:01 submitty CRON[19536]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Dec 13 00:08:29 submitty sshd[19552]: Invalid user shalini from 27.128.173.81 port 49454',
 'Dec 13 00:08:29 submitty sshd[19552]: pam_unix(sshd:auth): check pass; user unknown',
 'Dec 13 00:08:29 s

In [5]:
auth_log1 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/auth_log/auth.log.1")
#auth_log1.take(5)

In [6]:
auth_log2 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/auth_log/auth.log.2")
#auth_log2.take(5)

In [7]:
auth_log3 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/auth_log/auth.log.3")
#auth_log3.take(5)

In [8]:
auth_log4 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/auth_log/auth.log.4")
#auth_log4.take(5)

## 1. How many failed attempts to access the server as root are there? List all countries from which these attempts where carried out.

In [45]:
# Number of failed attempts to login as root
failed_attempts = auth_log.filter(lambda x: "root" in x and ("authentication failure" in x or "Failed password" in x or "session opened" not in x or "session closed" not in x))
#failed_attempts.take(50)
# find distinct attempts by unique ip
failed_attempts.count()

7899

In [52]:
# Countries where the failed attempts were carried out
# Split each literal to isolate the ip
# use python library to match ip to country
countries = failed_attempts.map(lambda line: line.rsplit("from"))
countries.take(5)

[['Dec 13 00:06:16 submitty sshd[19509]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.184  user=root'],
 ['Dec 13 00:06:17 submitty sshd[19509]: Failed password for root ',
  ' 218.92.0.184 port 3753 ssh2'],
 ['Dec 13 00:06:21 submitty sshd[19509]: Failed password for root ',
  ' 218.92.0.184 port 3753 ssh2'],
 ['Dec 13 00:09:01 submitty CRON[19580]: pam_unix(cron:session): session opened for user root by (uid=0)'],
 ['Dec 13 00:09:01 submitty CRON[19580]: pam_unix(cron:session): session closed for user root']]

## 2. How many failed attempts to access the server as non-root users are there? What are the attempted usernames? List all countries from which these attempts where carried out. 

In [47]:
# Number of failed attempts to login as non-root
failed_attempts_nonroot = auth_log.filter(lambda x: "root" not in x and ("authentication failure" not in x or "Failed password" not in x or "session opened" in x or "session closed" not in x))
#failed_attempts.take(5)
failed_attempts_nonroot.count()

# 

26016

## 3. Through out the durations of the log files, which date has the highest number of attack attempts? Anything interesting about that particular date?

In [ ]:
# Something to do with Chinese Labor Day (May 1)